In [22]:
import os, mariadb, json

from pipeline_funcs import read_last_line, met_past_24h_extract_list

In [9]:
# Verify connection with mariadb is working
try:
    with mariadb.connect(host = 'localhost',
                 user = os.getenv('MYSQL_USER'),
                 password = os.getenv('MYSQL_PASS'),
                 database = 'weather_comp') as connection:
        print(connection)
except mariadb.Error as e:
    print(e)

<mariadb.connection connected to 'localhost' at 000001FB9C4D1590>


In [18]:
# Insert a single record into met_past_24h table

# pydt = datetime.datetime(2024, 8, 6, 9, 0)
# print(pydt)
# strtime = time.strftime('%Y-%m-%d %H:%M:%S', pydt)
# print(strtime)

try:
    with mariadb.connect(host = 'localhost',
                 user = os.getenv('MYSQL_USER'),
                 password = os.getenv('MYSQL_PASS'),
                 database = 'weather_comp') as connection:
        cur = connection.cursor()
        cur.execute("""INSERT INTO met_past_24h (location, date_time, temp, weather_code, precip_bool) 
                    VALUES (?, ?, ?, ?, ?);
                    """, ('CREDENHILL', '2024-08-02 12:00:00', '17.1', '1', 0))
        connection.commit()
except mariadb.Error as e:
    print(e)

print(cur)

<mariadb.cursor at 000001FB9C7230B0>


In [42]:
table_name = 'met_past_24h'
col_name_dict = {'met_past_24h' : ['(location, date_time, temp, weather_code, precip_bool)', '(?, ?, ?, ?, ?)']}
col_names = col_name_dict['met_past_24h']
print(col_names[0])
print(col_names[1])

extract_dict = {'met_past_24h' : met_past_24h_extract_list}
extract_func = extract_dict['met_past_24h']

# insert_query = """
# INSERT INTO met_past_24h (location, date_time, temp, weather_code, precip_bool)
# VALUES (?, ?, ?, ?, ?)
# """

insert_query = f"""
                INSERT INTO {table_name} {col_names[0]}
                VALUES {col_names[1]}
                """

json_line = read_last_line('data/met_past_24h.jsonl')
# new_values = met_past_24h_extract_list(json_line)
new_values = extract_dict['met_past_24h'](json_line)

print(new_values)

(location, date_time, temp, weather_code, precip_bool)
(?, ?, ?, ?, ?)
[['CREDENHILL', '2024-08-06 9:00:00', '17.1', '1', 0], ['CREDENHILL', '2024-08-06 10:00:00', '18.9', '3', 0], ['CREDENHILL', '2024-08-06 11:00:00', '20.1', '1', 0], ['CREDENHILL', '2024-08-06 12:00:00', '19.4', '3', 0], ['CREDENHILL', '2024-08-06 13:00:00', '21.1', '3', 0], ['CREDENHILL', '2024-08-06 14:00:00', '20.7', '7', 0], ['CREDENHILL', '2024-08-06 15:00:00', '22.3', '7', 0], ['CREDENHILL', '2024-08-06 16:00:00', '21.7', '7', 0], ['CREDENHILL', '2024-08-06 17:00:00', '20.8', '3', 0], ['CREDENHILL', '2024-08-06 18:00:00', '19.7', '3', 0], ['CREDENHILL', '2024-08-06 19:00:00', '18.4', '3', 0], ['CREDENHILL', '2024-08-06 20:00:00', '15.1', '0', 0], ['CREDENHILL', '2024-08-06 21:00:00', '12.6', '0', 0], ['CREDENHILL', '2024-08-06 22:00:00', '11.3', '0', 0], ['CREDENHILL', '2024-08-06 23:00:00', '11.1', '0', 0], ['CREDENHILL', '2024-08-07 0:00:00', '10.9', '2', 0], ['CREDENHILL', '2024-08-07 1:00:00', '10.2', '0', 

In [43]:
try:
    with mariadb.connect(host = 'localhost',
                 user = os.getenv('MYSQL_USER'),
                 password = os.getenv('MYSQL_PASS'),
                 database = 'weather_comp') as connection:
        cur = connection.cursor()
        cur.executemany(insert_query, new_values)
        connection.commit()
except mariadb.Error as e:
    print(e)

In [37]:
# Retrieve data from met_past_24h to check it was inserted correctly
try:
    with mariadb.connect(host = 'localhost',
                 user = os.getenv('MYSQL_USER'),
                 password = os.getenv('MYSQL_PASS'),
                 database = 'weather_comp') as connection:
        cur = connection.cursor()
        cur.execute('SELECT * FROM met_past_24h;')
except mariadb.Error as e:
    print(e)

for (location, date_time, temp, weather_code, precip_bool) in cur:
    print((location, date_time, temp, weather_code, precip_bool))

('CREDENHILL', datetime.datetime(2024, 8, 2, 11, 0), 17.1, 1, 0)
('CREDENHILL', datetime.datetime(2024, 8, 2, 12, 0), 17.1, 1, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 9, 0), 17.1, 1, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 10, 0), 18.9, 3, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 11, 0), 20.1, 1, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 12, 0), 19.4, 3, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 13, 0), 21.1, 3, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 14, 0), 20.7, 7, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 15, 0), 22.3, 7, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 16, 0), 21.7, 7, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 17, 0), 20.8, 3, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 18, 0), 19.7, 3, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 19, 0), 18.4, 3, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 20, 0), 15.1, 0, 0)
('CREDENHILL', datetime.datetime(2024, 8, 6, 21, 0), 12.6, 0, 0)
('CREDENHILL', datetime.da

In [28]:
# catch up mariadb with json data collected so far
file_path = 'data/met_past_24h.jsonl'

with open(file_path, 'rb') as f:
    jsons = f.readlines()
    print(jsons)
    for j in jsons:
        print(json.loads(j))


[b'{"SiteRep": {"Wx": {"Param": [{"name": "G", "units": "mph", "$": "Wind Gust"}, {"name": "T", "units": "C", "$": "Temperature"}, {"name": "V", "units": "m", "$": "Visibility"}, {"name": "D", "units": "compass", "$": "Wind Direction"}, {"name": "S", "units": "mph", "$": "Wind Speed"}, {"name": "W", "units": "", "$": "Weather Type"}, {"name": "P", "units": "hpa", "$": "Pressure"}, {"name": "Pt", "units": "Pa/s", "$": "Pressure Tendency"}, {"name": "Dp", "units": "C", "$": "Dew Point"}, {"name": "H", "units": "%", "$": "Screen Relative Humidity"}]}, "DV": {"dataDate": "2024-08-02T12:00:00Z", "type": "Obs", "Location": {"i": "3522", "lat": "52.083", "lon": "-2.8", "name": "CREDENHILL", "country": "ENGLAND", "continent": "EUROPE", "elevation": "75.0", "Period": [{"type": "Day", "value": "2024-08-01Z", "Rep": [{"D": "NW", "G": "7", "H": "56.9", "P": "1013", "S": "3", "T": "25.2", "V": "27000", "W": "1", "Pt": "S", "Dp": "16.1", "$": "720"}, {"D": "N", "G": "10", "H": "59.4", "P": "1013", "